## Training Monitoring
Da nach dem Anstoß zum Training recht wenig Kontrolle über den Prozess. Hier werden Funktionen vorgestellt, diesen Prozess besser zu beobachten und zu steuern.

Bisher wurde der Trainingsprozess so vorgestellt, dass so viele Epochen trainiert werden sollte, bis das Modell anfängt zu overfitten. Wann der Trainingsprozess endet lässt sich auch über Callbacks steuern.

Callbacks sind Objekte, die bestimmte Funktionen implementieren und anderen Objekten übergeben werden, um sie situativ aufzurufen und so dem User zu ermöglichen, einen gegebenen Prozess anzupassen, indem er verschiedene Callbacks verwendet.

Nachfolgend wird gezeigt, wie sie verwendet werden und eine paar Callbacks vorgestellt.

In [ ]:
from tensorflow.keras import callbacks

callbacks_list = [
    callbacks.EarlyStopping( # unterbricht Training, wenn sich Metrik nicht mehr verbessert für [patience] Epochen
        monitor = 'acc',
        patience = 1
    ),
    callbacks.ModelCheckpoint( # Saves weights every epoch
        filepath = 'my_model.h5',
        monitor = 'val_loss',
        save_best_only = True
    ),
    callbacks.ReduceROnPleateau( # reduziert Lernrate auf 10%
        monitor = 'val_loss',
        factor = 0.1,
        patience = 10
    )
]

model.compile(
    optimizer = 'rmsprop',
    loss = 'binary_crossentropy',
    metrics = ['acc'] # Sollte hier sein, wenn durch Callback beobachtet
)

model.fit(
    X, y,
    epochs = 10,
    batch_size = 32,
    callback = callbacks_list,
    validation_data = (X_val, y_val)
)

Es lassen sich auch komplett eigene Callbacks definieren, was hier vorgestellt wird:

In [ ]:
import tensorflow.keras

class TestCallback(keras.callbacks.Callback): # erweitere gegebene Keras-Klasse
    def set_model(self, model):
        self.model = model # Called before training to inform callback what model it will be called by
        layer_outputs = [layer.output for layer in model.layers]
        self.activations_model = keras.models.Model(model.input, layer_outputs)
        
    # alle implementierbaren Methoden sind selbsterklärend und hier aufgelistet
    def on_epoch_end(self, epoch, logs = None):
        pass
    def on_epoch_begin(self, epoch, logs = None):
        pass
    
    
    def on_batch_begin(self, epoch, logs = None):
        pass
    def on_batch_end(self, epoch, logs = None):
        pass
    
    
    def on_train_begin(self):
        pass
    def on_train_end(self):
        pass

## Trainingsvisualisierung mit TensorBoard
TensorBoard kommt mit TensorFlow und ist browserbasiert. Es stellt eine große Menge an Visualisierungsmöglichkeiten für das Modell und sein Training bereit. Wir demonstrieren an dem einfachen IMDB-Beispiel:

In [7]:
from tensorflow.keras import layers
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
import tensorflow.keras as keras

max_features = 2000
max_len = 500

(train_X, train_y), (test_X, test_y) = imdb.load_data(num_words = max_features)
train_X = sequence.pad_sequences(train_X, maxlen = max_len)
test_X = sequence.pad_sequences(test_X, maxlen = max_len)

model = keras.models.Sequential()
model.add(layers.Embedding(
    max_features, 128,
    input_length = max_len,
    name = 'embed'
))
model.add(layers.Conv1D(32, 7, activation = 'relu'))
model.add(layers.MaxPooling1D(5))
model.add(layers.Conv1D(32, 7, activation = 'relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(1))
print(model.summary())

model.compile(
    optimizer = 'rmsprop',
    loss = 'binary_crossentropy',
    metrics = ['acc']
)

# wichtig, es sollte ein Ordner für das Monitoring angelegt werden
# Nun Tensorboard callback:
callbacks = [
    keras.callbacks.TensorBoard(
        log_dir = 'training_monitoring',
        histogram_freq = 1,
        embeddings_freq = 1
    )
]

history = model.fit(
    train_X, train_y,
    epochs = 20,
    batch_size = 128,
    validation_split = 0.2,
    callbacks = callbacks
)

# start TensorBoard server with tensorboard --logdir=training_monitoring

W0811 16:39:14.030402 140513352116032 deprecation.py:323] From /home/dominik/.local/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embed (Embedding)            (None, 500, 128)          256000    
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 494, 32)           28704     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 98, 32)            0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 92, 32)            7200      
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 291,937
Trainable params: 291,937
Non-trainable params: 0
________________________________________________

In [15]:
from tensorflow.keras.utils import plot_model
plot_model(model, to_file = 'model.png')

Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.


## Further Performance Enhancing Methods
### Batch Normalization
Sind viele Methoden, die die Beobachtungen einander angleichen wollen. Zum Beispiel die Normalisierung bei Normalverteilung, sodass $\mu = 0$ und $\sigma = 1$.
Normalisierung sollte auch zwischen den Schichten stattfinden, da es keine Garantie gibt, dass Daten, die das Ergebnis einer Schicht und normalisierten Inputdaten sind auch normal sind. Daher gibt es eine Schicht für diese Aufgabe: `BatchNormalization`. Diese Schicht hilft außerdem, die Updates der Backpropagation weiterzutragen, was ein tieferes Netzwerk ermöglicht.

In [ ]:
# ...
model.add(BatchNormalization())
# ...

### Batch Renormalization
Ist eine Weiterentwicklung, die bisher keine Nachteile aufweist. Es gibt auch Entwicklungen, die die Daten über das gesamte Netzwerk normal halten, indem spezielle Aktivierungsfunktionen und Initialisierungen genutzt werden.

### Depthwise Seperable Convolution
Ist eine Möglichkeit, die Convolutionschichten, schneller, leichter und präziser zu machen und kann mit dem `SeperableConv2D`-Konstruktor genutzt werden. Kurzgesagt nutzt sie die Convolution-Operation auf jedem Kanal des Bildes seperat.

### Hyperparameter Optimization
Hyperparameter bezeichnet Dinge wie die Tiefe des Netzwerks, ob `BatchNormalization`-Schichten genutzt werden, oder welche Aktivierungsfunktion eingsetzt wird. In der Praxis bauen Machine Learning Engineers mit der Zeit eine Intuition für diese Entscheidungen auf, aber wie kann hier ein Anfänger begründete Entscheidungen treffen?
Der typische Prozess läuft meist wiefolgt ab:
1. Wahl der anzupassenden Hyperparameter
2. Bauen des Modells
3. Fitting des Modells und Messen der Performanz anhand der Validierungsdaten
4. Wähle Hyperparameter neu
5. Wiederholde, bis eine gute Auswahl an Modellen gefunden ist
6. Teste die besten Modelle anhand der Testdaten.

Viele andere Möglichkeiten sind bekannt: genetische Algorithmen, Bayesianische Optimierung, Zufall, etc.
Hier ist allerdings wichtig, dass Schritt 4 in Abhängigkeit der vorangegangen Parameter gewählt wird. Denn bewegt sich die Performanz in eine gute Richtung, ist das Optimum vielleicht in dieser Richtung zu finden.
Gute Ansätze für automatische Wahl von Hyperparametern werden in den Repos von `Hyperopt` und `Hyperas` implementiert.

### Model Ensembling
Das ist eine Strategie, bei der viele Modelle eine gepoolte Voraussage treffen und ein in Wettbewerben sehr erfolgreicher Ansatz ist. Hier sind mehrere Ansätze den Durchschnitt einer Voraussage zu wählen oder ein gewichtetes Mittel, wobei die Gewichte von der Performanz auf die Validierungsdaten abhängig sind. Der Schlüssel hier die Unterschiedlichkeit der Modelle, während sie trotzdem einzeln gute Ergebnisse liefern. So können sie verschiedene Machine-Learning-Algorithmen sein oder komplett verschiedene Architekturen von Deep Learning-Modellen. Baumbasierte Methoden und NNs scheinen hierbei besonders gut zusammen zu funktionieren.